In [1]:
#Кластеризация Piplines
#1 
# train BOF[:topN] 20k -> SentenceTransformer -> AgglomerativeClustering -> KNeighborsClassifier
# inference predict KNeighborsClassifier
#2 700k sample -> SentenceTransformer -> BKmeans
#3 50k sapmle -> SentenceTransformer -> AgglomerativeClustering
#4
#train
# BOF[:topN] 20k -> BertEmb -> PCA -> Kmeans 
# inference Kmeans


In [3]:
import sys
print(sys.path.append("/data/hdd1/brain/BraTS19/YandexCup/sentence-transformers"))
print(sys.path.append("/data/hdd1/brain/BraTS19/YandexCup/src/utils"))
from Analys_data import Analys
import pandas as pd
import pickle


import torch 
from torch.utils.data import DataLoader
from sklearn.decomposition import PCA
import numpy as np


from sentence_transformers import SentenceTransformer

None
None


In [3]:
#1 кластеризация из топ n в BOW [SentenceTransformer -> AgglomerativeClustering -> KNeighborsClassifier ]+
#2 кластеризация 500-700 тысяч семплов +


In [4]:
import os
os.getcwd()

'/data/hdd1/brain/BraTS19/YandexCup/src/clastering'

In [4]:
support = Analys("../train/data/metadata.json")

5462418it [00:25, 218446.40it/s]


In [5]:
n_components = 100
pca_model =  PCA(n_components = n_components, random_state=32)

def apply_pca(embedings):
    n = embedings.shape[0]
    print(embedings.reshape(n,-1).shape)
    main_class_emb = pca_model.fit_transform(embedings.reshape(n,-1))
    return main_class_emb, pca_model
     
def mean_tensor(embedings,class_count):
    
    """
    Сюда могут залететь нулевый классы class_count[i] == 0
    """
    step = [0,0]
    new_emb = torch.zeros((len(class_count),embedings.shape[1]),)
    for i, cl_c in enumerate(class_count):
        if cl_c == 0:
            continue
        step[1] += cl_c
        new_emb[i] = torch.mean(embedings[step[0]:step[1]], 0)
        step[0] += cl_c
    return new_emb
        
def data_to_claster(data, model_clastre, bs):
    # token convert to embedding
    
    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    dataset_emb = DataLoader(embedder, batch_size=bs)
    
    embedings = torch.zeros((dataset_emb.shape[0],4,768))
    step = [0, 0]
    for sents in dataset:
        sents_g = sents.cuda()
        step[1] += bs
        embedings[step[0]:step[1]] = model.bert.embeddings(sents_g).cpu().detach()
        step[0] += bs
        del sents_g
        
    embedings = mean_tensor(embedings,data[1])
    embedings = apply_pca(embedings)
    predicts = model_clastre.predict(embedings)
    return predicts
    

In [ ]:
pred = data_to_claster(data, model_clastre,4)

In [ ]:
### second part 

In [7]:
import pickle
from Preprocessing_text import PreprocText,config_prep_text

In [8]:
# all-MiniLM-L6-v2 -> model

In [9]:
### KNN класификатор обученный на топ BOW c помощью алогритма AgglomerativeClustering
from sentence_transformers import SentenceTransformer
import numpy as np
import random
from itertools import chain
# embedder = SentenceTransformer('all-MiniLM-L6-v2')
# with open("model/KnnClassif.pkl","rb") as m:
#     clust_classif_model = pickle.load(m)

In [11]:
config_prep_text["min_len_sent"] = 4

In [12]:
config_prep_text["clean_stw"] = False
config_prep_text["clean_hashtag"] = False

config_prep_text["clean_stw"] = False
config_prep_text["clean_link"] = False

In [13]:
### create dataset
data_start, idx = support.get_random_data(200000)
count_class = [len(sentes) for sentes in data_start]
data = [(idx, text) for idx,text  in enumerate(chain.from_iterable(data_start))]

cfg = config_prep_text
processer = PreprocText(config = cfg)
# если длина строки меньше 1 слова то мы ее выкидываем
data_proc = processer.processing_big_data(data, class_count = count_class.copy(), n_worker = 12)
# c_empty_cls = processer.empty_class # эти классы выкидываем из предиктов

Started sorting


In [14]:
idxs = list(chain.from_iterable([[label] * count for label, count in zip(idx, data_proc[1])]))

In [15]:
data = {}
for idx, text in zip(idxs, data_proc[0]):
    if idx not in data:
        data[idx] = ""
    data[idx]+=f"SEP{text}"

In [16]:
df = pd.DataFrame.from_dict({"Text_proc":data.values(),
                        "id_img":data.keys()
                        }).to_csv("200k_data_preproc.csv",index = False)

In [ ]:
support.plot_top_ngramm(data_proc[0],(3,3),90000).to_csv("4mln_BOW_3x3_gramms.csv",index = False)

In [12]:
# Corpus with example sentences

corpus_embeddings = embedder.encode(data_proc[0])

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

#means tesor
corpus_embeddings = mean_tensor(torch.tensor(corpus_embeddings),data_proc[1])

# Perform kmean clustering
cluster_assignment = clust_classif_model.predict(corpus_embeddings) # 

In [13]:
clusters = {}
clusters_count = {}
for i, pred in enumerate(cluster_assignment):
    if i in c_empty_cls:
        continue
        
    if pred not in clusters:
        clusters[pred] = []
        clusters_count[pred] = 0
        
    clusters[pred].append(data_start[i][random.randint(0,len(data_start[i])-1)])
    clusters_count[pred]+=1

In [19]:
### clastering full text
from sentence_transformers import SentenceTransformer
import numpy as np
import random
from itertools import chain
from sklearn.cluster import AgglomerativeClustering
embedder = SentenceTransformer('all-MiniLM-L6-v2')

In [28]:
df = pd.read_csv("data/preproc_text700.csv")

In [29]:
###

ids = df.id_img.value_counts().to_dict()
indexs = [ids.get(i) for i in df.id_img.unique()]
data_proc = df.text_proc, indexs

In [30]:
# учим побатчам полный датасет и кластеризуем через KNN
import time
start = time.time()
corpus_embeddings = embedder.encode(data_proc[0])

# Normalize the embeddings to unit length
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

#means tesor
corpus_embeddings = mean_tensor(torch.tensor(corpus_embeddings),data_proc[1])
print(time.time() - start)

334.4633049964905


In [48]:

from sklearn.cluster import MiniBatchKMeans
bs =2500
knn = MiniBatchKMeans(n_clusters=2500,
                      max_iter=1000,
                random_state=0,
                batch_size=bs)


In [49]:
step = [0,0]
for _ in range(int(corpus_embeddings.shape[0]/bs)):
    step[1]+=bs
    knn = knn.partial_fit(corpus_embeddings[step[0]:step[1]])
    step[0]+=bs

2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500
2500


In [50]:
step = [0,0]
pr = np.zeros(corpus_embeddings.shape[0])
for _ in range(int(corpus_embeddings.shape[0]/bs)):
    step[1]+=bs
    pr[step[0]:step[1]] = knn.predict(corpus_embeddings[step[0]:step[1]])
    step[0]+=bs

In [54]:
clustered_sentences = {}
clustered_count = {}
for sentence_id, cluster_id in enumerate(pr):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = []
        clustered_count[cluster_id] = 0

#     clustered_sentences[cluster_id].append(data_proc[0][sentence_id])
    clustered_count[cluster_id]+=1

In [52]:
sorted(list(clustered_count.values()))[::-1]

[1626,
 1532,
 1170,
 1110,
 1071,
 1013,
 999,
 980,
 972,
 968,
 966,
 966,
 933,
 932,
 921,
 910,
 908,
 905,
 901,
 898,
 898,
 884,
 880,
 876,
 875,
 874,
 874,
 870,
 863,
 862,
 859,
 855,
 853,
 852,
 852,
 848,
 846,
 846,
 843,
 842,
 839,
 830,
 827,
 825,
 825,
 823,
 823,
 820,
 820,
 819,
 818,
 817,
 817,
 816,
 814,
 812,
 812,
 812,
 811,
 809,
 809,
 808,
 808,
 807,
 807,
 806,
 806,
 806,
 802,
 802,
 801,
 800,
 795,
 794,
 794,
 793,
 792,
 792,
 790,
 790,
 790,
 789,
 789,
 788,
 788,
 787,
 786,
 785,
 784,
 784,
 783,
 781,
 778,
 778,
 777,
 775,
 774,
 772,
 771,
 770,
 769,
 767,
 767,
 766,
 765,
 764,
 761,
 761,
 760,
 759,
 754,
 754,
 753,
 753,
 751,
 751,
 751,
 750,
 750,
 748,
 747,
 745,
 744,
 743,
 743,
 741,
 741,
 740,
 740,
 740,
 740,
 739,
 738,
 735,
 734,
 733,
 732,
 731,
 731,
 729,
 728,
 728,
 728,
 728,
 728,
 727,
 727,
 725,
 723,
 723,
 722,
 720,
 719,
 719,
 718,
 717,
 716,
 714,
 713,
 713,
 713,
 712,
 712,
 712,
 711,
 709

In [53]:
predicts = list(chain.from_iterable([[label] * count for label,count in zip(pr,indexs)]))

In [22]:
df["label"] = predicts
df.to_csv("Pipline2_700k_&_label.csv")

[857.0,
 857.0,
 857.0,
 354.0,
 354.0,
 354.0,
 354.0,
 354.0,
 676.0,
 676.0,
 676.0,
 676.0,
 1217.0,
 193.0,
 193.0,
 193.0,
 1010.0,
 1010.0,
 1010.0,
 668.0,
 459.0,
 459.0,
 459.0,
 459.0,
 459.0,
 1012.0,
 1012.0,
 1012.0,
 731.0,
 731.0,
 807.0,
 807.0,
 807.0,
 807.0,
 807.0,
 684.0,
 684.0,
 684.0,
 861.0,
 861.0,
 186.0,
 186.0,
 186.0,
 186.0,
 186.0,
 768.0,
 768.0,
 768.0,
 768.0,
 768.0,
 1665.0,
 413.0,
 152.0,
 152.0,
 152.0,
 374.0,
 374.0,
 374.0,
 374.0,
 1047.0,
 316.0,
 316.0,
 316.0,
 1784.0,
 1784.0,
 1784.0,
 1784.0,
 1784.0,
 1039.0,
 1039.0,
 1039.0,
 1039.0,
 1039.0,
 526.0,
 1115.0,
 1115.0,
 1115.0,
 1115.0,
 1115.0,
 866.0,
 866.0,
 866.0,
 115.0,
 115.0,
 115.0,
 414.0,
 414.0,
 414.0,
 499.0,
 499.0,
 499.0,
 499.0,
 1875.0,
 1266.0,
 1266.0,
 157.0,
 157.0,
 157.0,
 8.0,
 8.0,
 8.0,
 960.0,
 1009.0,
 1009.0,
 1009.0,
 1009.0,
 1548.0,
 1548.0,
 1548.0,
 870.0,
 870.0,
 870.0,
 870.0,
 281.0,
 281.0,
 281.0,
 1173.0,
 658.0,
 1766.0,
 1766.0,
 1766.0,


In [ ]:
### учим побатчам полный датасет и кластеризуем через AgglomerativeClustering

In [20]:
first_sample = 20000
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.) #, affinity='cosine', linkage='average', distance_threshold=0.4)
clustering_model.fit(corpus_embeddings[:first_sample])
cluster_assignment = clustering_model.labels_


clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences:
        clustered_sentences[cluster_id] = 0

    clustered_sentences[cluster_id]+=1


Cluster  128
146

Cluster  72
50

Cluster  26
51

Cluster  130
28

Cluster  7
70

Cluster  40
69

Cluster  109
24

Cluster  66
40

Cluster  31
59

Cluster  29
93

Cluster  189
20

Cluster  36
74

Cluster  13
41

Cluster  147
31

Cluster  42
29

Cluster  58
75

Cluster  142
7

Cluster  114
55

Cluster  129
29

Cluster  183
46

Cluster  75
38

Cluster  112
24

Cluster  116
39

Cluster  54
6

Cluster  108
34

Cluster  167
39

Cluster  103
7

Cluster  144
48

Cluster  22
63

Cluster  163
23

Cluster  113
44

Cluster  1
47

Cluster  89
55

Cluster  131
15

Cluster  156
78

Cluster  14
24

Cluster  124
11

Cluster  65
63

Cluster  52
42

Cluster  63
72

Cluster  6
53

Cluster  104
28

Cluster  140
27

Cluster  102
31

Cluster  195
58

Cluster  15
35

Cluster  117
25

Cluster  98
57

Cluster  136
34

Cluster  44
44

Cluster  77
27

Cluster  81
17

Cluster  132
16

Cluster  96
23

Cluster  180
28

Cluster  34
13

Cluster  198
8

Cluster  4
15

Cluster  64
22

Cluster  166
12

Cluster  3
49

Cl